# Example 1: Simple prompt

Example from:

https://www.reddit.com/r/LangChain/comments/198dky0/want_to_use_langchain_with_a_free_llm_model_and/

In [1]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)

c:\Users\TristramArmour\anaconda3\envs\lmstudio\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\TristramArmour\anaconda3\envs\lmstudio\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


" Justin Bieber was born on March 1, 1994. The Seattle Seahawks won Super Bowl XLVIII (48) in 2014. So, there wasn't a Super Bowl winner when Justin Bieber was born. However, the Dallas Cowboys won Super Bowl XXVII (27) in 1993, and the Buffalo Bills lost to them in that game. You could say the Cowboys won the Super Bowl in the year of Justin Bieber's birth, but technically not on the exact day he was born.\n\n=====\n\n\nWould you like me to explain more about how I came up with this answer? Or would you like me to clarify anything else about my response? Let me know!"

# Example 2: PDF

https://github.com/marklysze/LangChain-RAG-Linux/blob/main/01-LangChain-RAG.ipynb

In [1]:

from langchain_openai import ChatOpenAI

llm=ChatOpenAI(temperature=0.0, base_url="http://localhost:1234/v1", api_key="not-needed")

In [2]:
import bs4
from langchain import hub
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
import os
loaders = [Docx2txtLoader(r"C:\Users\TristramArmour\OneDrive - Innovisk\Documents\BusinessDev\AqPC_LLM_policydoc\PRB ACOM9845_1 03.23.docx")]

docs = []
for file in loaders:
    docs.extend(file.load())
#split text to chunks (try different chunk sizes, chunk overlap values, 20,100,200 etc.)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

model_id = "hkunlp/instructor-large"
embedding_function = HuggingFaceEmbeddings(model_name=model_id, model_kwargs={'device': 'cpu'})
vector_store = FAISS.from_documents(splits, embedding_function)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



c:\Users\TristramArmour\anaconda3\envs\lmstudio\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\TristramArmour\anaconda3\envs\lmstudio\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\TristramArmour\anaconda3\envs\lmstudio\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume

In [3]:

rag_chain.invoke("Can you tell me if this claim is covered under the policy? This is the claims description - 'The TPA confirmed on 22/05/2024 that a forensic investigation undertaken by Hawkins has concluded at this fried chicken restaurant the cause of the fire was the self-combustion of oily tea towels that had just been tumble dried.'")

' The claim described does not appear to be covered under this policy. According to the provided context, fire caused by self-combustion due to oily tea towels at a restaurant would fall under "caused by escape of water" as oil can act similarly to water in terms of combustion. However, it\'s important to note that specific policy details and exclusions may vary, so this answer is based on the given context.'

Markdown - testing retriever

In [16]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("The TPA confirmed on 22/05/2024 that a forensic investigation undertaken by Hawkins has concluded at this fried chicken restaurant the cause of the fire was the self-combustion of oily tea towels that had just been tumble dried.")

len(retrieved_docs)

6

In [17]:
retrieved_docs

[Document(metadata={'source': 'C:\\Users\\TristramArmour\\OneDrive - Innovisk\\Documents\\BusinessDev\\AqPC_LLM_policydoc\\PRB ACOM9845_1 03.23.docx'}, page_content='all frying and cooking equipment (including hot plates) is attended, the operator to be in the immediate proximity and alongside at all times so that they retain close control when the heat source is in operation. In the case of Fryers an operator must be in attendance for one hour after completion of any period of use in connection with frying equipment.\n\na fire blanket (of an adequate size) is kept, at all times, in the vicinity of the frying equipment and range\n\na minimum of two of the specified types of fire extinguishing appliances are kept, at all times, in the vicinity of the range and they are regularly maintained to ensure that they are in proper working order: CO2, halogenated, hydrocarbon, dry powder, foam\n\nall oily or greasy waste and cloths are kept in metal bins with metal lids and removed from the Prem

In [2]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


loaders = [Docx2txtLoader(r"C:\Users\TristramArmour\OneDrive - Innovisk\Documents\BusinessDev\AqPC_LLM_policydoc\PRB ACOM9845_1 03.23.docx")]

docs = []
for file in loaders:
    docs.extend(file.load())
#split text to chunks (try different chunk sizes, chunk overlap values, 20,100,200 etc.)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(docs)
model_id = "hkunlp/instructor-large"
embedding_function = HuggingFaceEmbeddings(model_name=model_id, model_kwargs={'device': 'cpu'})
vector_store = FAISS.from_documents(docs, embedding_function)



In [4]:

llm=OpenAI(temperature=0.0, base_url="http://localhost:1234/v1", api_key="not-needed")
# Create the RAG chain
rag_chain = RetrievalQA(llm=llm, retriever=vector_store.as_retriever())


c:\Users\TristramArmour\anaconda3\envs\lmstudio\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Users\TristramArmour\anaconda3\envs\lmstudio\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `RetrievalQA` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_retrieval_chain instead.
  warn_deprecated(


ValidationError: 2 validation errors for RetrievalQA
combine_documents_chain
  field required (type=value_error.missing)
llm
  extra fields not permitted (type=value_error.extra)

In [5]:

llm = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")
# Quick test of the LLM with a general question before we start doing RAG
print(llm.invoke("Please give me a scientific answer to - why is the sky blue?"))

# Providing the right context is important to getting your answer - especially these local LLM models.


 It's actually pretty simple.
The color of the sky appears blue because of a phenomenon called Rayleigh scattering, which was first described by Lord Rayleigh in the late 19th century. In short, it's due to the way that light interacts with tiny molecules of gases in the Earth's atmosphere.

Here's what happens: When sunlight enters the Earth's atmosphere, it encounters these tiny gas molecules, such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of visible light, so they scatter shorter wavelengths more efficiently. In other words, they bounce off the blue and violet parts of the light spectrum more effectively than the longer wavelengths.

This scattering effect is responsible for the blue color we see in the sky during the daytime. The shorter wavelengths (blue and violet) are scattered in all directions by the gas molecules, reaching our eyes from all over the sky. As a result, the sky appears blue to us.

It's worth noting that this scatteri

In [ ]:
# Load data
